In [1]:
import pandas as pd
import numpy as np
import tkinter as tk
import calendar
import warnings
import locale
import os
from tkinter import messagebox
from datetime import datetime

warnings.filterwarnings('ignore')
locale.setlocale(locale.LC_TIME, 'es_ES.UTF-8')

def obtener_fecha(mes_anterior=False):
    hoy = datetime.now()
    
    if mes_anterior:
        if hoy.month == 1:
            mes = 12
            año = hoy.year - 1
        else:
            mes = hoy.month - 1
            año = hoy.year
    else:
        mes = hoy.month
        año = hoy.year
    
    mes_nombre = calendar.month_abbr[mes].upper()[:3] # ENE
    mes_año = f"{mes_nombre}{str(año)[2:]}" # ENE24
    fecha = f"{año}{str(mes).zfill(2)}" # 202401
    
    return mes_año, fecha

root = tk.Tk()
root.attributes('-topmost', True)
root.withdraw()

result = messagebox.askquestion('Confirmación', '¿Cargar mes anterior?', icon='warning')
if result == 'yes':
    mes_año, fecha = obtener_fecha(mes_anterior=True)
else:
    mes_año, fecha = obtener_fecha()
root.destroy()

efectividades_path = f'bases/efectividades/{fecha}/Efectividades_{fecha}.xlsx'
dashboard_efect_path = f'dashboard/{fecha}/DASHBOARD_EFECTIVIDADES_{mes_año}.xlsx'
dashboard_saldos_path = f'dashboard/{fecha}/DASHBOARD_SALDOS_{mes_año}.xlsx'
dashboard_indicadores_path = f'dashboard/{fecha}/DASHBOARD_INDICADORES_{mes_año}.xlsx'

dashboard_efect_path = os.path.abspath(dashboard_efect_path)
dashboard_saldos_path = os.path.abspath(dashboard_saldos_path)
dashboard_indicadores_path = os.path.abspath(dashboard_indicadores_path)

print(efectividades_path)
print(dashboard_efect_path)
print(dashboard_saldos_path)
print(dashboard_indicadores_path)

bases/efectividades/202501/Efectividades_202501.xlsx
c:\Users\p042833\Documents\GitHub\base-pagos\dashboard\202501\DASHBOARD_EFECTIVIDADES_ENE25.xlsx
c:\Users\p042833\Documents\GitHub\base-pagos\dashboard\202501\DASHBOARD_SALDOS_ENE25.xlsx
c:\Users\p042833\Documents\GitHub\base-pagos\dashboard\202501\DASHBOARD_INDICADORES_ENE25.xlsx


In [2]:
def clean_columns(columns_list: list[str]) -> list[str]:
    return [column.strip().replace('.', '').replace(' ', '_').upper() for column in columns_list]

In [3]:
def obtener_periodos_año(año: int) -> list[str]:
    periodos = []
    for mes in range(1, 13):
        fecha = f"{año}{str(mes).zfill(2)}" # 202401, 202402, etc.
        periodos.append(fecha)
    return periodos

def leer_archivos_csv(periodos: list[str]) -> dict[str, pd.DataFrame]:
    dataframes = {}
    for periodo in periodos:
        file_path = f'bases/efectividades/{periodo}/Efectividades_{periodo}.csv'
        if os.path.exists(file_path):
            df = pd.read_csv(file_path, sep=';', encoding='utf-8')
            print(f"Periodo: {periodo} - {df.shape}")
            dataframes[periodo] = df
        else:
            print(f"Archivo no encontrado: {file_path}")
    return dataframes

In [4]:
def dashboard_efectividades(df_pagos_efect: pd.DataFrame) -> pd.DataFrame:
    df_pagos_efect.columns = clean_columns(df_pagos_efect.columns)
    cols =  ['PERIODO', 'AGENCIA', 'CLAVE', 'FOCO', 'CAPITALSOLES', 'PAGOEFECTTOTALSOLESAGENCIACONT']
    df_pagos_efect = df_pagos_efect[cols]
    print(df_pagos_efect.shape)
    
    df_pagos_efect.rename(columns={
        'CLAVE': 'CARTERA', 
        'FOCO': 'TRAMO',
        'CAPITALSOLES': 'CAPITAL', 
        'PAGOEFECTTOTALSOLESAGENCIACONT': 'RECUPERO', 
    }, inplace=True)
    
    df_pagos_efect = df_pagos_efect[df_pagos_efect['AGENCIA'].isin(['ASESCOM RJ', 'CLASA MORA', 'MORNESE MORA'])]
    df_pagos_efect['CAPITAL'] = df_pagos_efect['CAPITAL'].round(2)
    df_pagos_efect['RECUPERO'] = df_pagos_efect['RECUPERO'].round(2)
    df_pagos_efect['TRAMO'] = df_pagos_efect['TRAMO'].fillna('NULL')
    
    df_pagos_efect = df_pagos_efect.groupby(['PERIODO', 'AGENCIA', 'CARTERA', 'TRAMO']).agg({'RECUPERO': 'sum', 'CAPITAL': 'sum'}).reset_index()
    
    return df_pagos_efect

In [5]:
def dashboard_saldos(df_pagos_saldos: pd.DataFrame) -> pd.DataFrame:
    df_pagos_saldos.columns = clean_columns(df_pagos_saldos.columns)
    cols =  ['PERIODO', 'AGENCIA', 'CLAVE', 'FOCO', 'CAPITALSOLES', 'SEGMENTO_RIESGO', 'AMBITO_RCD_FINAL']
    df_pagos_saldos = df_pagos_saldos[cols]
    print(df_pagos_saldos.shape)
    
    df_pagos_saldos.rename(columns={
        'CLAVE': 'CARTERA', 
        'FOCO': 'TRAMO',
        'CAPITALSOLES': 'CAPITAL', 
        'SEGMENTO_RIESGO': 'SEGMENTO', 
        'AMBITO_RCD_FINAL': 'PRODUCTO' 
    }, inplace=True)
    
    df_pagos_saldos = df_pagos_saldos[df_pagos_saldos['AGENCIA'].isin(['ASESCOM RJ', 'CLASA MORA', 'MORNESE MORA', 'SIN AGENCIA'])]
    
    df_pagos_saldos['SEGMENTO'] = df_pagos_saldos['SEGMENTO'].apply(lambda x: 'PYME / EMP MIN' if x != 'PARTICULARES' else x)
    df_pagos_saldos['PRODUCTO'] = df_pagos_saldos['PRODUCTO'].astype(str).str.replace(r'\d+', '', regex=True).str.replace('.', '').str.upper()
    df_pagos_saldos['PRODUCTO'] = df_pagos_saldos['PRODUCTO'].astype(str).str.strip().replace(' ', '')
    df_pagos_saldos['CAPITAL'] = df_pagos_saldos['CAPITAL'].round(2)
    df_pagos_saldos['TRAMO'] = df_pagos_saldos['TRAMO'].fillna('NULL')
    
    df_pagos_saldos = df_pagos_saldos.groupby(['PERIODO', 'AGENCIA', 'CARTERA', 'TRAMO', 'SEGMENTO', 'PRODUCTO']).agg({'CAPITAL': 'sum'}).reset_index()
    
    return df_pagos_saldos

In [6]:
def dashboard_indicadores(df_pagos_indicadores: pd.DataFrame) -> pd.DataFrame:
    df_pagos_indicadores.columns = clean_columns(df_pagos_indicadores.columns)
    cols =  ['PERIODO', 'AGENCIA', 'CODCENT', 'CLAVE', 'FOCO', 'INTENSIDAD', 'DIRECTO_CALL', 'COBERTURA', 'CONTACTO_EFECTIVO', 'TASA_CIERRE', 'PDP_CUMPLIDA']
    df_pagos_indicadores = df_pagos_indicadores[cols]
    print(df_pagos_indicadores.shape)
    
    df_pagos_indicadores.rename(columns={
        'CLAVE': 'CARTERA', 
        'FOCO': 'TRAMO', 
        'INTENSIDAD': 'INTENSIDAD_TOTAL', 
        'DIRECTO_CALL': 'INTENSIDAD_DIRECTA', 
        'CONTACTO_EFECTIVO': 'CONTACTO_DIRECTO', 
        'PDP_CUMPLIDA': 'CALIDAD_PROMESAS'
        }, inplace=True)
    
    df_pagos_indicadores = df_pagos_indicadores[df_pagos_indicadores['AGENCIA'].isin(['ASESCOM RJ', 'CLASA MORA', 'MORNESE MORA'])]
    
    df_pagos_indicadores['INTENSIDAD_TOTAL'] = df_pagos_indicadores['INTENSIDAD_TOTAL'].fillna(0)
    df_pagos_indicadores['INTENSIDAD_DIRECTA'] = df_pagos_indicadores['INTENSIDAD_DIRECTA'].fillna(0)
    df_pagos_indicadores['COBERTURA'] = df_pagos_indicadores['COBERTURA'].fillna(0)
    df_pagos_indicadores['CONTACTO_DIRECTO'] = df_pagos_indicadores['CONTACTO_DIRECTO'].fillna('NULL')
    df_pagos_indicadores['TASA_CIERRE'] = df_pagos_indicadores['TASA_CIERRE'].fillna('NULL')
    df_pagos_indicadores['CALIDAD_PROMESAS'] = df_pagos_indicadores['CALIDAD_PROMESAS'].fillna('NULL')
    df_pagos_indicadores['TRAMO'] = df_pagos_indicadores['TRAMO'].fillna('NULL')
    # convierte 'NULL' en NaN
    df_pagos_indicadores['COBERTURA'] = pd.to_numeric(df_pagos_indicadores['COBERTURA'], errors='coerce')
    df_pagos_indicadores['CONTACTO_DIRECTO'] = pd.to_numeric(df_pagos_indicadores['CONTACTO_DIRECTO'], errors='coerce')
    df_pagos_indicadores['TASA_CIERRE'] = pd.to_numeric(df_pagos_indicadores['TASA_CIERRE'], errors='coerce')
    df_pagos_indicadores['CALIDAD_PROMESAS'] = pd.to_numeric(df_pagos_indicadores['CALIDAD_PROMESAS'], errors='coerce')
    
    
    df_validados_intensidad = df_pagos_indicadores.groupby(['CARTERA', 'TRAMO']).agg(
        CONTEO_DIFERENCIADO=('CODCENT', 'nunique'),
        CONTEO_DIFERENCIADO_CD=('CODCENT', lambda x: x[df_pagos_indicadores['INTENSIDAD_DIRECTA'] != 0].nunique()), 
    ).reset_index()
    
    df_pagos_indicadores = df_pagos_indicadores.groupby(['PERIODO', 'AGENCIA', 'CARTERA', 'TRAMO']).agg({
        'INTENSIDAD_TOTAL': 'sum', 
        'INTENSIDAD_DIRECTA': 'sum', 
        'COBERTURA': 'mean', 
        'CONTACTO_DIRECTO': lambda x: x.mean(skipna=True), 
        'TASA_CIERRE': lambda x: x.mean(skipna=True), 
        'CALIDAD_PROMESAS': lambda x: x.mean(skipna=True)
    }).reset_index()
    
    df_pagos_indicadores = pd.merge(df_pagos_indicadores, df_validados_intensidad, on=['CARTERA', 'TRAMO'], how='left')
    df_pagos_indicadores['INTENSIDAD_TOTAL'] = df_pagos_indicadores['INTENSIDAD_TOTAL']/df_pagos_indicadores['CONTEO_DIFERENCIADO']
    df_pagos_indicadores['INTENSIDAD_DIRECTA'] = df_pagos_indicadores['INTENSIDAD_DIRECTA']/df_pagos_indicadores['CONTEO_DIFERENCIADO_CD']
    df_pagos_indicadores.drop(columns=['CONTEO_DIFERENCIADO', 'CONTEO_DIFERENCIADO_CD'], inplace=True)
    
    df_pagos_indicadores['INTENSIDAD_TOTAL'] = df_pagos_indicadores['INTENSIDAD_TOTAL'].round(2)
    df_pagos_indicadores['INTENSIDAD_DIRECTA'] = df_pagos_indicadores['INTENSIDAD_DIRECTA'].round(2)
    df_pagos_indicadores['COBERTURA'] = df_pagos_indicadores['COBERTURA'].round(4)
    df_pagos_indicadores['CONTACTO_DIRECTO'] = df_pagos_indicadores['CONTACTO_DIRECTO'].round(2)
    df_pagos_indicadores['TASA_CIERRE'] = df_pagos_indicadores['TASA_CIERRE'].round(2)
    df_pagos_indicadores['CALIDAD_PROMESAS'] = df_pagos_indicadores['CALIDAD_PROMESAS'].round(2)
    
    df_pagos_indicadores.sort_values(by=['PERIODO', 'AGENCIA', 'CARTERA', 'TRAMO'], inplace=True)
    
    return df_pagos_indicadores

In [7]:
root = tk.Tk()
root.attributes('-topmost', True)
root.withdraw()

result = messagebox.askquestion('Confirmación', '¿Cargar efectividades?', icon='warning')
if result == 'yes':
    df_pagos = pd.read_excel(efectividades_path)
    print('Base Efectividades:', df_pagos.shape)

root.destroy()

In [ ]:
df_pagos_test = df_pagos.copy()
df_pagos_test.columns = clean_columns(df_pagos_test.columns)

df_pagos_efect = df_pagos_test.copy()
df_pagos_saldos = df_pagos_test.copy()
df_pagos_indicadores = df_pagos_test.copy()

In [ ]:
df_pagos_efect = dashboard_efectividades(df_pagos_efect)
df_pagos_efect.to_excel(dashboard_efect_path, index=False)
print(df_pagos_efect.shape)
df_pagos_efect.head()

In [ ]:
df_pagos_saldos = dashboard_saldos(df_pagos_saldos)
df_pagos_saldos.to_excel(dashboard_saldos_path, index=False)
print(df_pagos_saldos.shape)
df_pagos_saldos.head()

In [ ]:
df_pagos_indicadores = dashboard_indicadores(df_pagos_indicadores)
df_pagos_indicadores.to_excel(dashboard_indicadores_path, index=False)
print(df_pagos_indicadores.shape)
df_pagos_indicadores.head()

In [ ]:
os.startfile(dashboard_efect_path)
os.startfile(dashboard_saldos_path)
os.startfile(dashboard_indicadores_path)

In [8]:
root = tk.Tk()
root.attributes('-topmost', True)
root.withdraw()

result = messagebox.askquestion('Confirmación', '¿Año anterior?', icon='warning')
if result == 'yes':
    año = datetime.now().year - 1
else:
    año = datetime.now().year
root.destroy()

resultado_path = f'dashboard/Resultados_{año}.xlsx'
resultado_path

'dashboard/Resultados_2025.xlsx'

In [ ]:
root = tk.Tk()
root.attributes('-topmost', True)
root.withdraw()

result = messagebox.askquestion('Confirmación', f'¿Cargar año {año}?', icon='warning')
if result == 'yes':
    periodos = obtener_periodos_año(año)
    dataframes = leer_archivos_csv(periodos)
    
    resultado = {
        "Efectividades": pd.DataFrame(),
        "Saldos": pd.DataFrame(),
        "Indicadores": pd.DataFrame()
    }
    
    for periodo, df in dataframes.items():
        df_efect = dashboard_efectividades(df.copy())
        df_saldos = dashboard_saldos(df.copy())
        df_indicadores = dashboard_indicadores(df.copy())
        
        resultado["Efectividades"] = pd.concat([resultado["Efectividades"], df_efect], ignore_index=True)
        resultado["Saldos"] = pd.concat([resultado["Saldos"], df_saldos], ignore_index=True)
        resultado["Indicadores"] = pd.concat([resultado["Indicadores"], df_indicadores], ignore_index=True)
    
    with pd.ExcelWriter(resultado_path) as writer:
        resultado["Efectividades"].to_excel(writer, sheet_name='Efectividades', index=False)
        resultado["Saldos"].to_excel(writer, sheet_name='Saldos', index=False)
        resultado["Indicadores"].to_excel(writer, sheet_name='Indicadores', index=False)
    
    os.startfile(os.path.abspath(resultado_path))

root.destroy()